# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [35]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [36]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kawambwa,-9.7915,29.0791,11.83,44,26,2.21,ZM,1687140045
1,1,saint-philippe,-21.3585,55.7679,19.77,80,26,4.37,RE,1687140045
2,2,olonkinbyen,70.9221,-8.7187,3.40,99,100,2.63,SJ,1687140046
3,3,alofi,-19.0595,-169.9187,25.94,83,10,4.63,NU,1687140046
4,4,pont rouge,48.8833,-72.0825,14.16,82,100,2.57,CA,1687140046


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    scale = 0.75,
    color="City",
    cmap='glasbey_light', 
    frame_width = 700,
    frame_height = 600,
    alpha=0.5,
    title="City Humidity Map"

)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [51]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Humidity"] < 55) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
criteria_cities_df = criteria_cities_df.dropna()

# Display sample data
criteria_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,amursk,50.2261,136.8994,24.98,47,0,1.85,RU,1687140050
129,129,hit,33.6416,42.8251,26.64,30,0,4.02,IQ,1687140077
206,206,sakakah,29.9697,40.2064,26.76,21,0,2.65,SA,1687140096
238,238,dalnerechensk,45.9339,133.7225,24.82,54,0,1.13,RU,1687140103
306,306,al badari,26.9926,31.4155,26.08,38,0,3.39,EG,1687140121
420,420,suwayr,30.1171,40.3893,26.52,21,0,3.27,SA,1687140151
444,444,awjilah,29.1081,21.2869,21.69,47,0,3.86,LY,1687140157
492,492,siwa oasis,29.2041,25.5195,22.89,49,0,2.80,EG,1687140168
575,575,kearney,40.6995,-99.0815,26.95,54,0,3.60,US,1687140001


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[
    ["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kawambwa,ZM,-9.7915,29.0791,44,
1,saint-philippe,RE,-21.3585,55.7679,80,
2,olonkinbyen,SJ,70.9221,-8.7187,99,
3,alofi,NU,-19.0595,-169.9187,83,
4,pont rouge,CA,48.8833,-72.0825,82,
...,...,...,...,...,...,...
594,zaza del medio,CU,21.9961,-79.3669,89,
595,al kharijah,EG,25.4514,30.5464,29,
596,popondetta,PG,-8.7537,148.2534,54,
597,lazaro cardenas,MX,17.9583,-102.2000,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kawambwa - nearest hotel: No hotel found
saint-philippe - nearest hotel: Le Baril
olonkinbyen - nearest hotel: No hotel found
alofi - nearest hotel: Matavai Resort
pont rouge - nearest hotel: No hotel found
qaqortoq - nearest hotel: Hotel Qaqortoq
margaret river - nearest hotel: Margaret River Hotel
vernon - nearest hotel: Okanagan Royal Park Inn
ribeira grande - nearest hotel: Hospedaria JSF - 3 Estrelas
codrington - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
dunedin - nearest hotel: Scenic Hotel Dunedin City
punta arenas - nearest hotel: Hotel Lacolet
grytviken - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
adamstown - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
amursk - nearest hotel: Центральная
ust-nera - nearest hotel: гостиница "Солнечная"
mil'kovo - nearest hotel: Дрозды
georgetown - nearest hotel: Page 63 host

dwarka - nearest hotel: The Dwarika Hotel
middelburg - nearest hotel: No hotel found
mugia - nearest hotel: Solpor do Coído
micheweni - nearest hotel: No hotel found
zheleznogorsk-ilimskiy - nearest hotel: Магнетит
suhbaatar - nearest hotel: Selenge Hotel
klyuchi - nearest hotel: No hotel found
huarmey - nearest hotel: Hostal Santa Rosa
el bauga - nearest hotel: No hotel found
kitui - nearest hotel: Parkside Hotel
ghadamis - nearest hotel: فندق عين الفارس
la passe - nearest hotel: L'Estuaire
invercargill - nearest hotel: The Grand
zhangjiakou - nearest hotel: 商务宾馆
playa del ingles - nearest hotel: Apartamentos Barbados
fernie - nearest hotel: Royal Hotel
villa gesell - nearest hotel: Hotel El Faro
sint michiel liber - nearest hotel: Blue Bay Lodges
caleta de carquin - nearest hotel: Hotel Mi Choacan
sakakah - nearest hotel: فندق النزل
chonchi - nearest hotel: Hotel Huildín
amanpur - nearest hotel: No hotel found
oke ila - nearest hotel: No hotel found
priargunsk - nearest hotel: No hot

broome - nearest hotel: No hotel found
sanctuary point - nearest hotel: Dungowan Accommodation
myitkyina - nearest hotel: New Light Hotel
lucapa - nearest hotel: No hotel found
puerto quijarro - nearest hotel: No hotel found
fandriana - nearest hotel: Hotel Sariaka
touros - nearest hotel: Pousada Atlântico
chipinge - nearest hotel: No hotel found
tondano - nearest hotel: Hotel Tou Dano
san juan bautista - nearest hotel: Hacienda San Juan Resort & Spa
ocean shores - nearest hotel: Collins Inn & Seaside Cottages
neiafu - nearest hotel: Port Wine Guest House
thabazimbi - nearest hotel: Maroela Guest Lodge
arkalyk - nearest hotel: Гостиница "Арсенал"
garbahaarrey - nearest hotel: No hotel found
mariental - nearest hotel: Mariental Hotel
cabedelo - nearest hotel: No hotel found
najaf - nearest hotel: مديرية مجاري النجف
east ballina - nearest hotel: Ballina Beach Resort
ixtapa - nearest hotel: Ma. Cristina
aoulef - nearest hotel: دار الضياف
bel air riviere seche - nearest hotel: Otentic Eco 

abepura - nearest hotel: Yoyo
panacan - nearest hotel: Gorayan Lodge
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
kearney - nearest hotel: Days Inn
wulingyuan - nearest hotel: 张家界武陵居客栈
nouadhibou - nearest hotel: Hotel valencia
colima - nearest hotel: No hotel found
dudhi - nearest hotel: No hotel found
lerwick - nearest hotel: Grand Hotel
tobelo - nearest hotel: JJ Guesthouse
lubuklinggau - nearest hotel: Cozy Style
chimbote - nearest hotel: Hostal Diamante Chimbote
port moresby - nearest hotel: The Sanctuary Hotel and Spa
hermantown - nearest hotel: Holiday Inn Express & Suites Duluth North - Miller Hill, an IHG Hotel
batangafo - nearest hotel: No hotel found
janoshalma - nearest hotel: No hotel found
mopti - nearest hotel: No hotel found
taulaga - nearest hotel: No hotel found
greymouth - nearest hotel: Recreation Hotel
ariquemes - nearest hotel: Aquarius
vilyuysk - nearest hotel: Мария
yuscaran - nearest hotel: Hotel Yuscarán
zaza del medio - nearest hotel: Islaz

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kawambwa,ZM,-9.7915,29.0791,44,No hotel found
1,saint-philippe,RE,-21.3585,55.7679,80,Le Baril
2,olonkinbyen,SJ,70.9221,-8.7187,99,No hotel found
3,alofi,NU,-19.0595,-169.9187,83,Matavai Resort
4,pont rouge,CA,48.8833,-72.0825,82,No hotel found
...,...,...,...,...,...,...
594,zaza del medio,CU,21.9961,-79.3669,89,Islazul Villa Los Laureles
595,al kharijah,EG,25.4514,30.5464,29,Kharga Hotel
596,popondetta,PG,-8.7537,148.2534,54,No hotel found
597,lazaro cardenas,MX,17.9583,-102.2000,81,Hotel Sol del Pacífico


In [17]:
# Save Data to csv
hotel_df.to_csv("output_data/hotels.csv", index=False)

In [18]:
# Read saved data
hotel_df = pd.read_csv("output_data/hotels.csv")

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kawambwa,ZM,-9.7915,29.0791,44,No hotel found
1,saint-philippe,RE,-21.3585,55.7679,80,Le Baril
2,olonkinbyen,SJ,70.9221,-8.7187,99,No hotel found
3,alofi,NU,-19.0595,-169.9187,83,Matavai Resort
4,pont rouge,CA,48.8833,-72.0825,82,No hotel found
...,...,...,...,...,...,...
594,zaza del medio,CU,21.9961,-79.3669,89,Islazul Villa Los Laureles
595,al kharijah,EG,25.4514,30.5464,29,Kharga Hotel
596,popondetta,PG,-8.7537,148.2534,54,No hotel found
597,lazaro cardenas,MX,17.9583,-102.2000,81,Hotel Sol del Pacífico


In [29]:
# Drop rows with "No hotel found" in the "Hotel Name" column
hotel_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]

# Display the updated DataFrame
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,saint-philippe,RE,-21.3585,55.7679,80,Le Baril
3,alofi,NU,-19.0595,-169.9187,83,Matavai Resort
5,qaqortoq,GL,60.7167,-46.0333,96,Hotel Qaqortoq
6,margaret river,AU,-33.9500,115.0667,68,Margaret River Hotel
7,vernon,CA,50.2581,-119.2691,52,Okanagan Royal Park Inn
...,...,...,...,...,...,...
593,yuscaran,HN,13.9333,-86.8500,96,Hotel Yuscarán
594,zaza del medio,CU,21.9961,-79.3669,89,Islazul Villa Los Laureles
595,al kharijah,EG,25.4514,30.5464,29,Kharga Hotel
597,lazaro cardenas,MX,17.9583,-102.2000,81,Hotel Sol del Pacífico


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    color="City",
    cmap='glasbey_light', 
    frame_width = 700,
    frame_height = 600,
    alpha=0.5,
    title="Hotel Locations Map",
    hover_cols=["Country", "Hotel Name"]

)


# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)